In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from kneed import KneeLocator
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

In [2]:
os.chdir("F:\\Road to Opta\\player_stats\\")
football_dataset = pd.read_csv("football_scoreline.csv")
football_dataset = football_dataset.iloc[:,1:]

In [3]:
columns = football_dataset.columns.to_list()

In [4]:
columns = columns[0:1] + columns[7:8] + columns[9:95]

In [5]:
df = football_dataset[[*columns]]
position = pd.get_dummies(df.pos,prefix="pos")

In [6]:
df = df.drop(['pos'],axis=1)

In [7]:
df = df.join(position)

In [8]:
df.head(10)

,player_name,min,st,on,off,y,yr,r,g,a,...,pos_DMR,pos_DR,pos_FW,pos_FWL,pos_FWR,pos_GK,pos_MC,pos_ML,pos_MR,pos_SUB
0,Pepe,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,Nacho,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,Keylor Navas,90,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,Kiko Casilla,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,Luka Modric,90,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5,Lucas Vazquez,90,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Mateo Kovacic,86,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Isco,66,1,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Casemiro,24,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
9,James Rodriguez,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [9]:
from sklearn.preprocessing import StandardScaler
scaled_features = StandardScaler().fit_transform(df.iloc[:,1:].values)
scaled_features_df = pd.DataFrame(scaled_features, index=df.iloc[:,1:].index, columns=df.iloc[:,1:].columns)
scaled_features_df.head(5)

,min,st,on,off,y,yr,r,g,a,sa,...,pos_DMR,pos_DR,pos_FW,pos_FWL,pos_FWR,pos_GK,pos_MC,pos_ML,pos_MR,pos_SUB
0,-1.276538,-1.147475,-0.441596,-0.441773,-0.335785,-0.048814,-0.070528,-0.244355,-0.212275,-0.072646,...,-0.038519,-0.204207,-0.274384,-0.105629,-0.105616,-0.233395,-0.305633,-0.162946,-0.162972,1.147518
1,-1.276538,-1.147475,-0.441596,-0.441773,-0.335785,-0.048814,-0.070528,-0.244355,-0.212275,-0.072646,...,-0.038519,-0.204207,-0.274384,-0.105629,-0.105616,-0.233395,-0.305633,-0.162946,-0.162972,1.147518
2,0.974814,0.871479,-0.441596,-0.441773,-0.335785,-0.048814,-0.070528,-0.244355,-0.212275,-0.072646,...,-0.038519,-0.204207,-0.274384,-0.105629,-0.105616,4.284576,-0.305633,-0.162946,-0.162972,-0.871446
3,-1.276538,-1.147475,-0.441596,-0.441773,-0.335785,-0.048814,-0.070528,-0.244355,-0.212275,-0.072646,...,-0.038519,-0.204207,-0.274384,-0.105629,-0.105616,-0.233395,-0.305633,-0.162946,-0.162972,1.147518
4,0.974814,0.871479,-0.441596,-0.441773,-0.335785,-0.048814,-0.070528,-0.244355,4.127387,-0.072646,...,-0.038519,-0.204207,-0.274384,-0.105629,-0.105616,-0.233395,-0.305633,-0.162946,-0.162972,-0.871446


In [10]:
scaled_features_df = pd.concat([df.player_name,scaled_features_df],axis=1)
scaled_features_df.head(5)

,player_name,min,st,on,off,y,yr,r,g,a,...,pos_DMR,pos_DR,pos_FW,pos_FWL,pos_FWR,pos_GK,pos_MC,pos_ML,pos_MR,pos_SUB
0,Pepe,-1.276538,-1.147475,-0.441596,-0.441773,-0.335785,-0.048814,-0.070528,-0.244355,-0.212275,...,-0.038519,-0.204207,-0.274384,-0.105629,-0.105616,-0.233395,-0.305633,-0.162946,-0.162972,1.147518
1,Nacho,-1.276538,-1.147475,-0.441596,-0.441773,-0.335785,-0.048814,-0.070528,-0.244355,-0.212275,...,-0.038519,-0.204207,-0.274384,-0.105629,-0.105616,-0.233395,-0.305633,-0.162946,-0.162972,1.147518
2,Keylor Navas,0.974814,0.871479,-0.441596,-0.441773,-0.335785,-0.048814,-0.070528,-0.244355,-0.212275,...,-0.038519,-0.204207,-0.274384,-0.105629,-0.105616,4.284576,-0.305633,-0.162946,-0.162972,-0.871446
3,Kiko Casilla,-1.276538,-1.147475,-0.441596,-0.441773,-0.335785,-0.048814,-0.070528,-0.244355,-0.212275,...,-0.038519,-0.204207,-0.274384,-0.105629,-0.105616,-0.233395,-0.305633,-0.162946,-0.162972,1.147518
4,Luka Modric,0.974814,0.871479,-0.441596,-0.441773,-0.335785,-0.048814,-0.070528,-0.244355,4.127387,...,-0.038519,-0.204207,-0.274384,-0.105629,-0.105616,-0.233395,-0.305633,-0.162946,-0.162972,-0.871446


In [11]:
import sqlite3
os.chdir("F:\\Road to Opta\\player_stats\\")
cnn=sqlite3.connect('game_data.db')
scaled_features_df.to_sql('scaled_features',cnn)

In [12]:
cursor = cnn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('scaled_features',)]


In [13]:
#cursor.execute("DROP TABLE scaled_features")

In [14]:
%reload_ext sql

In [15]:
%sql sqlite:///game_data.db

In [16]:
## Mean stats of every player

mean_stats_query = '''
Select player_name,
AVG(min) as min_mean,
AVG(st) as st_mean,
AVG('on') as on_mean,
AVG(off) as off_mean,
AVG(y) as y_mean,
AVG(yr) as yr_mean,
AVG(r) as r_mean,
AVG(g) as g_mean,
AVG(a) as a_mean,
AVG(sa) as sa_mean,
AVG(aop) as aop_mean,
AVG(asp) as asp_mean,
AVG(s) as s_mean,
AVG(sog) as sog_mean,
AVG(int) as int_mean,
AVG(cr) as cr_mean,
AVG(acr) as acr_mean,
AVG(cc) as cc_mean,
AVG(bcc) as bcc_mean,
AVG(ccop) as ccop_mean,
AVG(ccsp) as ccsp_mean,
AVG(blk) as blk_mean,
AVG(tkl) as tkl_mean,
AVG(tklw) as tklw_mean,
AVG(fc) as fc_mean,
AVG(fs) as fs_mean,
AVG(p) as p_mean,
AVG(ap) as ap_mean,
AVG(acro) as acro_mean,
AVG(aw) as aw_mean,
AVG(br) as br_mean,
AVG(dr) as dr_mean,
AVG(dw) as dw_mean,
AVG(eg) as eg_mean,
AVG(es) as es_mean,
AVG(ibs) as ibs_mean,
AVG(ibsog) as ibsog_mean,
AVG(ibg) as ibg_mean,
AVG(obs) as obs_mean,
AVG(obsog) as obsog_mean,
AVG(obg) as obg_mean,
AVG(aog) as aog_mean,
AVG(apw) as apw_mean,
AVG(dsp) as dsp_mean,
AVG(own) as own_mean,
AVG(touch) as touch_mean,
AVG(tbox) as tbox_mean,
AVG(pft) as pft_mean,
AVG(dl) as dl_mean,
AVG(afzp) as afzp_mean,
AVG(blkcr) as blkcr_mean,
AVG(pent) as pent_mean,
AVG(clrol) as clrol_mean,
AVG(posl) as posl_mean,
AVG(lpopp) as lpopp_mean,
AVG(pk) as pk_mean,
AVG(pkg) as pkg_mean,
AVG(pkm) as pkm_mean,
AVG(pksvd) as pksvd_mean,
AVG(fkcr) as fkcr_mean,
AVG(fkacr) as fkacr_mean,
AVG(crn) as crn_mean,
AVG(crncr) as crncr_mean,
AVG(crnw) as crnw_mean,
AVG(fks) as fks_mean,
AVG(fksog) as fksog_mean,
AVG(fkg) as fkg_mean,
AVG(gc) as gc_mean,
AVG(cs) as cs_mean,
AVG(sv) as sv_mean,
AVG(ibsv) as ibsv_mean,
AVG(obsv) as obsv_mean,
AVG(aks) as aks_mean,
AVG(pkc) as pkc_mean,
AVG(pkf) as pkf_mean,
AVG(pksv) as pksv_mean,
AVG(cl) as cl_mean,
AVG(ecl) as ecl_mean,
AVG(punch) as punch_mean,
AVG(alb) as alb_mean,
AVG(atb) as atb_mean,
AVG(lmt) as lmt_mean,
AVG(toff) as toff_mean,
AVG(bcm) as bcm_mean,
AVG(bcs) as bcs_mean,
AVG(attdr) as attdr_mean,
AVG(pos_AMC) as pos_AMC_mean,
AVG(pos_AML) as pos_AML_mean,
AVG(pos_AMR) as pos_AMR_mean,
AVG(pos_DC) as pos_DC_mean,
AVG(pos_DL) as pos_DL_mean,
AVG(pos_DMC) as pos_DMC_mean,
AVG(pos_DML) as pos_DML_mean,
AVG(pos_DMR) as pos_DMR_mean,
AVG(pos_DR) as pos_DR_mean,
AVG(pos_FW) as pos_FW_mean,
AVG(pos_FWL) as pos_FWL_mean,
AVG(pos_FWR) as pos_FWR_mean,
AVG(pos_GK) as pos_GK_mean,
AVG(pos_MC) as pos_MC_mean,
AVG(pos_ML) as pos_ML_mean,
AVG(pos_MR) as pos_MR_mean,
AVG(pos_SUB) as pos_SUB_mean from scaled_features
group by player_name
'''

In [17]:
mean_stats = %sql $mean_stats_query

 * sqlite:///game_data.db
Done.


In [19]:
col = scaled_features_df.columns

In [21]:
mean_stats = pd.DataFrame(mean_stats,columns=col)

# Cosine Similarity

In [24]:
import numpy as np; import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
cosine = cosine_similarity(mean_stats.iloc[:,1:])

In [135]:
cosine_df = pd.DataFrame(cosine, index=mean_stats.iloc[:,1:].index)
cosine_df = pd.concat([mean_stats.player_name,cosine_df],axis=1)
cosine_df.head(5)

,player_name,0,1,2,3,4,5,6,7,8,...,8262,8263,8264,8265,8266,8267,8268,8269,8270,8271
0,Aaron,1.000000,-0.088614,-0.230908,0.710104,0.041530,0.561088,0.356518,0.041530,0.035899,...,0.467070,-0.304772,0.207063,0.041530,0.041530,0.143231,-0.180956,0.041530,0.041530,-0.127775
1,Aaron Appindangoye,-0.088614,1.000000,-0.421192,0.204318,-0.272126,-0.022129,-0.209671,-0.272126,-0.217335,...,0.090543,-0.187473,-0.132463,-0.272126,-0.272126,-0.201436,0.871253,-0.272126,-0.272126,0.215133
2,Aaron Connolly,-0.230908,-0.421192,1.000000,-0.556787,0.566677,0.143795,-0.251186,0.566677,0.347549,...,0.136646,0.296116,-0.401379,0.566677,0.566677,-0.130482,-0.469241,0.566677,0.566677,0.244661
3,Aaron Cresswell,0.710104,0.204318,-0.556787,1.000000,-0.498445,0.121537,0.551974,-0.498445,-0.308756,...,0.091514,-0.004361,0.568878,-0.498445,-0.498445,0.311941,0.189618,-0.498445,-0.498445,-0.392309
4,Aaron Herzog,0.041530,-0.272126,0.566677,-0.498445,1.000000,0.405237,-0.328694,1.000000,0.533110,...,0.381252,-0.400848,-0.588064,1.000000,1.000000,-0.259843,-0.329321,1.000000,1.000000,0.637603


In [136]:
all_players = cosine_df.player_name.to_list()
cosine_df.columns = ['player_name'] + all_players

In [187]:
similar_player = 'Leonardo Bonucci'

In [188]:
similar_df = cosine_df.loc[cosine_df['player_name'] == similar_player]
similar_df.set_index('player_name',inplace=True)

In [190]:
similar_df.sort_values(by = similar_player, axis=1, ascending=False).iloc[:,1:10]

,Raul Albiol,Kevin Vogt,Aymeric Laporte,Thiago Silva,Jerome Boateng,Manuel Akanji,Sven Botman,Ruben Dias,Aissa Mandi
player_name,,,,,,,,,
Leonardo Bonucci,0.963951,0.960401,0.959254,0.950515,0.948951,0.944626,0.94249,0.936173,0.935288


# Euclidean Distance

In [197]:
from scipy.spatial.distance import cdist

euclid = cdist(mean_stats.iloc[:,1:], mean_stats.iloc[:,1:], 'euclid')

In [198]:
euclid_df = pd.DataFrame(euclid, index=mean_stats.iloc[:,1:].index)
euclid_df = pd.concat([mean_stats.player_name,euclid_df],axis=1)
euclid_df.head(5)

,player_name,0,1,2,3,4,5,6,7,8,...,8262,8263,8264,8265,8266,8267,8268,8269,8270,8271
0,Aaron,0.000000,6.778065,5.298571,3.853736,5.627407,3.013169,6.957683,5.627407,4.285061,...,3.880141,10.545620,6.056035,5.627407,5.627407e+00,9.288906,5.901703,5.627407,5.627407e+00,4.510651
1,Aaron Appindangoye,6.778065,0.000000,7.743523,6.972125,8.254492,6.496951,10.206940,8.254492,6.841309,...,6.657192,11.527414,8.606711,8.254492,8.254492e+00,11.685751,2.824342,8.254492,8.254492e+00,5.691612
2,Aaron Connolly,5.298571,7.743523,0.000000,7.877487,3.946824,4.302541,8.926536,3.946824,3.629575,...,4.980470,8.685041,7.851731,3.946824,3.946824e+00,10.206568,6.679684,3.946824,3.946824e+00,3.812258
3,Aaron Cresswell,3.853736,6.972125,7.877487,0.000000,8.760700,5.913794,6.322400,8.760700,6.856143,...,6.486773,10.574885,5.222176,8.760700,8.760700e+00,9.078015,6.260851,8.760700,8.760700e+00,6.921067
4,Aaron Herzog,5.627407,8.254492,3.946824,8.760700,0.000000,4.459746,9.993074,0.000000,3.986180,...,4.914174,11.753772,9.368734,0.000000,1.207547e-15,11.332772,7.357894,0.000000,5.460275e-16,3.629761


In [199]:
all_players = euclid_df.player_name.to_list()
euclid_df.columns = ['player_name'] + all_players

In [202]:
similar_player = 'Lionel Messi'

In [205]:
similar_df = euclid_df.loc[euclid_df['player_name'] == similar_player]
similar_df.set_index('player_name',inplace=True)
similar_df.sort_values(by = similar_player, axis=1, ascending=True).iloc[:,11:20]

,Mislav Orsic,Iago Aspas,Mohamed Salah,Bruno Fernandes,Lorenzo Insigne,Kevin De Bruyne,Andrej Kramaric,Philippe Coutinho,Junior Messias
player_name,,,,,,,,,
Lionel Messi,12.515642,12.545196,12.628903,12.783109,12.809116,12.82434,12.991977,13.014109,13.040338


In [12]:
# import numpy as np; import pandas as pd
# from sklearn.metrics.pairwise import cosine_similarity
# cosine_similarity(scaled_features_df.iloc[:,1:])


In [ ]:
# kmeans = KMeans(
#     init="random",
#     n_clusters=10,
#     n_init=10,
#     max_iter=1)

In [ ]:
# kmeans.fit(scaled_features)

In [14]:
# import tensorflow as tf
# y_true = [[0., 1.], [1., 1.]]
# y_pred = [[1., 0.], [1., 1.]]
# # Using 'auto'/'sum_over_batch_size' reduction type.
# cosine_loss = tf.keras.losses.CosineSimilarity(axis=1)
# # l2_norm(y_true) = [[0., 1.], [1./1.414], 1./1.414]]]
# # l2_norm(y_pred) = [[1., 0.], [1./1.414], 1./1.414]]]
# # l2_norm(y_true) . l2_norm(y_pred) = [[0., 0.], [0.5, 0.5]]
# # loss = mean(sum(l2_norm(y_true) . l2_norm(y_pred), axis=1))
# #       = -((0. + 0.) +  (0.5 + 0.5)) / 2
# cosine_loss(y_true, y_pred).numpy()


-0.49999997